In [3]:
!pip install datasets nltk sklearn

In [54]:
!pip install datasets 

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
import datasets
import os
import pandas as pd
import re
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
def load_data(dataset_split='train'):
    dataset = datasets.load_dataset('rotten_tomatoes')[dataset_split]
    # Open and import positve data
    df = pd.DataFrame()
    df['Review'] = [review['text'] for review in dataset]
    df['Sentiment'] = [review['label'] for review in dataset]
    # Remove non-alphanumeric characters
    df['Review'] = df['Review'].apply(lambda x: re.sub("[^a-zA-Z]", ' ', str(x)))
    # Tokenize the training and testing data
    df_tokenized = tokenize_review(df)
    return df_tokenized

In [6]:
def tokenize_review(df):
    # Tokenize Reviews in training
    tokened_reviews = [word_tokenize(rev) for rev in df['Review']]
    # Create word stems
    stemmed_tokens = []
    porter = PorterStemmer()
    for i in range(len(tokened_reviews)):
        stems = [porter.stem(token) for token in tokened_reviews[i]]
        stems = ' '.join(stems)
        stemmed_tokens.append(stems)
    df.insert(1, column='Stemmed', value=stemmed_tokens)
    return df

# **Bag of Words (BOW) and Tf-idf unstemmed**





In [7]:
def transform_BOW(training, testing, column_name):
    vect = CountVectorizer(max_features=100, ngram_range=(1,3), stop_words=ENGLISH_STOP_WORDS)
    vectFit = vect.fit(training[column_name])
    BOW_training = vectFit.transform(training[column_name])
    BOW_training_df = pd.DataFrame(BOW_training.toarray(), columns=vect.get_feature_names())
    BOW_testing = vectFit.transform(testing[column_name])
    BOW_testing_Df = pd.DataFrame(BOW_testing.toarray(), columns=vect.get_feature_names())
    return vectFit, BOW_training_df, BOW_testing_Df

In [8]:
def transform_tfidf(training, testing, column_name):
    Tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=100, stop_words=ENGLISH_STOP_WORDS)
    Tfidf_fit = Tfidf.fit(training[column_name])
    Tfidf_training = Tfidf_fit.transform(training[column_name])
    Tfidf_training_df = pd.DataFrame(Tfidf_training.toarray(), columns=Tfidf.get_feature_names())
    Tfidf_testing = Tfidf_fit.transform(testing[column_name])
    Tfidf_testing_df = pd.DataFrame(Tfidf_testing.toarray(), columns=Tfidf.get_feature_names())
    return Tfidf_fit, Tfidf_training_df, Tfidf_testing_df

In [70]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV

# def build_model(X_train, y_train, X_test, y_test, name_of_test):
#     # log_reg = LogisticRegression(C=30, max_iter=200).fit(X_train, y_train)
#     linear_svm = svm.SVC(kernel='linear',C=1.0,probability=True)
#     linear_svm.fit(X_train, y_train)
#     # linearn_svm = LinearSVC(C=1)

#     y_pred = linear_svm.predict(X_test)
#     print('Training accuracy of '+name_of_test+': ', linear_svm.score(X_train, y_train))
#     print('Testing accuracy of '+name_of_test+': ', linear_svm.score(X_test, y_test))
#     print(classification_report(y_test, y_pred)) 
#     return linear_svm

def build_model(X_train, y_train, X_test, y_test, name_of_test):
    # log_reg = LogisticRegression(C=30, max_iter=200).fit(X_train, y_train)
    lin_svm = LinearSVC(C=1)
    linear_svm = CalibratedClassifierCV(lin_svm)
    linear_svm.fit(X_train, y_train)
    # linearn_svm = LinearSVC(C=1)
    y_pred = linear_svm.predict(X_test)
    print('Training accuracy of '+name_of_test+': ', linear_svm.score(X_train, y_train))
    print('Testing accuracy of '+name_of_test+': ', linear_svm.score(X_test, y_test))
    print(classification_report(y_test, y_pred)) 
    return linear_svm

**loading training and testing dataset**

In [59]:
df_train = load_data('train')
df_test = load_data('test')

df_train

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

,Review,Stemmed,Sentiment
0,the rock is destined to be the st century s ...,the rock is destin to be the st centuri s new ...,1
1,the gorgeously elaborate continuation of the...,the gorgeous elabor continu of the lord of the...,1
2,effective but too tepid biopic,effect but too tepid biopic,1
3,if you sometimes like to go to the movies to h...,if you sometim like to go to the movi to have ...,1
4,emerges as something rare an issue movie tha...,emerg as someth rare an issu movi that s so ho...,1
...,...,...,...
8525,any enjoyment will be hinge from a personal th...,ani enjoy will be hing from a person threshold...,0
8526,if legendary shlockmeister ed wood had ever ma...,if legendari shlockmeist ed wood had ever made...,0
8527,hardly a nuanced portrait of a young woman s b...,hardli a nuanc portrait of a young woman s bre...,0
8528,interminably bleak to say nothing of boring,intermin bleak to say noth of bore,0


# Bag of Words(BOW)

In [71]:
# Create unstemmed BOW features for training set
unstemmed_BOW_vect_fit, df_train_bow_unstem, df_test_bow_unstem = transform_BOW(df_train, df_test, 'Review')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# TF-idf

In [72]:
unstemmed_tfidf_vect_fit, df_train_tfidf_unstem, df_test_tfidf_unstem = transform_tfidf(df_train, df_test, 'Review')


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Training linear svm model on BOW(unstemmed) features and tf-idf(unstemmed) features 

In [73]:
bow_unstemmed = build_model(df_train_bow_unstem, df_train['Sentiment'], df_test_bow_unstem, df_test['Sentiment'], 'BOW Unstemmed')

Training accuracy of BOW Unstemmed:  0.6202813599062134
Testing accuracy of BOW Unstemmed:  0.6041275797373359
              precision    recall  f1-score   support

           0       0.59      0.69      0.63       533
           1       0.63      0.52      0.57       533

    accuracy                           0.60      1066
   macro avg       0.61      0.60      0.60      1066
weighted avg       0.61      0.60      0.60      1066



In [74]:
tfidf_unstemmed = build_model(df_train_tfidf_unstem, df_train['Sentiment'], df_test_tfidf_unstem, df_test['Sentiment'], 'TFIDF Unstemmed')

Training accuracy of TFIDF Unstemmed:  0.6213364595545134
Testing accuracy of TFIDF Unstemmed:  0.6078799249530957
              precision    recall  f1-score   support

           0       0.60      0.67      0.63       533
           1       0.62      0.54      0.58       533

    accuracy                           0.61      1066
   macro avg       0.61      0.61      0.61      1066
weighted avg       0.61      0.61      0.61      1066



# Installing textattack toolbox

In [26]:
!pip install textattack
!pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 5.1 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konoha 4.6.5 requires importlib-metadata<4.0.0,>=3.7.0, but you have importlib-metadata 4.8.2 which is incompatible.


# Performing Textfooler attack 

In [48]:
import textattack
from textattack.models.wrappers import SklearnModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker



**On Unstemmed BOW features**

In [75]:
model_wrapper = SklearnModelWrapper(bow_unstemmed, unstemmed_BOW_vect_fit)
dataset = HuggingFaceDataset("rotten_tomatoes", None, "train")
attack = TextFoolerJin2019.build(model_wrapper)


Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset rotten_tomatoes, split train.
textattack: Unknown if model of class <class 'sklearn.calibration.CalibratedClassifierCV'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [76]:
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:06<01:01,  6.83s/it]/usr/local/li

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (54%)]] --> [[Negative (51%)]]

the rock is destined to be the 21st century's [[new]] " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .

the rock is destined to be the 21st century's [[newest]] " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (52%)]] --> [[Negative (51%)]]

the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/[[director]] peter jackson's expanded vision of j . r . r . tolkien's middle-earth .

the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words c

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  30%|███       | 3/10 [00:07<00:16,  2.35s/it]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/util

--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (51%)]] --> [[[SKIPPED]]]

effective but too-tepid biopic




...

--------------------------------------------- Result 4 ---------------------------------------------
[[Positive (69%)]] --> [[Negative (61%)]]

if you sometimes like to go to the [[movies]] to have [[fun]] , wasabi is a good place to start .

if you sometimes like to go to the [[movie]] to have [[amuse]] , wasabi is a good place to start .


--------------------------------------------- Result 5 ---------------------------------------------
[[Negative (75%)]] --> [[[SKIPPED]]]

emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .




/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:  60%|██████    | 6/10 [00:07<00:05,  1.25s/it]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:  70%|███████   | 7/10 [00:07<00:03,  1.08s/it]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out inste

--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (63%)]] --> [[Negative (58%)]]

the [[film]] provides some [[great]] insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .

the [[movie]] provides some [[admirable]] insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .


--------------------------------------------- Result 7 ---------------------------------------------
[[Negative (51%)]] --> [[[SKIPPED]]]

offers that rare combination of entertainment and education .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (55%)]] --> [[Negative (51%)]]

perhaps no picture ever made has more literally showed that the road to hell is paved with [[good]] intentions .

perhaps no picture ever made has more literally showed that the road to hell is paved with [[

[Succeeded / Failed / Skipped / Total] 5 / 0 / 4 / 9:  90%|█████████ | 9/10 [00:07<00:00,  1.16it/s]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 5 / 0 / 5 / 10: 100%|██████████| 10/10 [00:07<00:00,  1.29it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (51%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (51%)]] --> [[[SKIPPED]]]

take care of my cat offers a refreshingly different slice of asian cinema .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 5      |
| Original accuracy:            | 50.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 6.08%  |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 67.6

**On Unstemmed tf-idf features**

In [77]:
from textattack.models.wrappers import SklearnModelWrapper

model_wrapper = SklearnModelWrapper(tfidf_unstemmed, unstemmed_tfidf_vect_fit)

In [78]:
dataset = HuggingFaceDataset("rotten_tomatoes", None, "train")
attack = TextFoolerJin2019.build(model_wrapper)


Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset rotten_tomatoes, split train.
textattack: Unknown if model of class <class 'sklearn.calibration.CalibratedClassifierCV'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [79]:
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:05<00:47,  5.25s/it]/usr/local/li

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (51%)]] --> [[Negative (53%)]]

the rock is destined to be the 21st century's [[new]] " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .

the rock is destined to be the 21st century's [[newest]] " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (52%)]] --> [[Negative (52%)]]

the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/[[director]] peter jackson's expanded vision of j . r . r . tolkien's middle-earth .

the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words c

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  30%|███       | 3/10 [00:05<00:12,  1.82s/it]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/util

--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (52%)]] --> [[[SKIPPED]]]

effective but too-tepid biopic




...

--------------------------------------------- Result 4 ---------------------------------------------
[[Positive (65%)]] --> [[Negative (61%)]]

if you sometimes like to go to the [[movies]] to have [[fun]] , wasabi is a good place to start .

if you sometimes like to go to the [[movie]] to have [[amuse]] , wasabi is a good place to start .


--------------------------------------------- Result 5 ---------------------------------------------
[[Negative (74%)]] --> [[[SKIPPED]]]

emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .




/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:  60%|██████    | 6/10 [00:05<00:03,  1.01it/s]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:  70%|███████   | 7/10 [00:05<00:02,  1.17it/s]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out inste

--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (69%)]] --> [[Negative (57%)]]

the [[film]] provides some [[great]] insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .

the [[movie]] provides some [[big]] insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .


--------------------------------------------- Result 7 ---------------------------------------------
[[Negative (52%)]] --> [[[SKIPPED]]]

offers that rare combination of entertainment and education .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (53%)]] --> [[Negative (52%)]]

perhaps no picture ever made has more literally showed that the road to hell is paved with [[good]] intentions .

perhaps no picture ever made has more literally showed that the road to hell is paved with [[decent

[Succeeded / Failed / Skipped / Total] 5 / 0 / 4 / 9:  90%|█████████ | 9/10 [00:06<00:00,  1.46it/s]/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[Succeeded / Failed / Skipped / Total] 5 / 0 / 5 / 10: 100%|██████████| 10/10 [00:06<00:00,  1.61it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (52%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (52%)]] --> [[[SKIPPED]]]

take care of my cat offers a refreshingly different slice of asian cinema .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 5      |
| Original accuracy:            | 50.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 6.08%  |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 66.8